In [ ]:
import os
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

IMAGE_EXT = ".png"

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20})

In [ ]:
dirpath = "/media/ml4c3/bedmaster_start_time_analysis"

In [ ]:
times = dict()

fpaths = []
for root, dirs, files in os.walk(dirpath):
    for fname in files:
        if fname.endswith(".csv"):
            fpath = os.path.join(root, fname)
            fpaths.append(fpath)

            floor = fpath.split("_")[-1].replace(".csv", "")
            df = pd.read_csv(fpath)
            times[floor] = (
                mdates.date2num(df[df.iloc[:, 0] == "BM"]["First"].values[0]),
                mdates.date2num(df[df.iloc[:, 0] == "BM"]["Last"].values[0]),
            )

In [ ]:
df_start_end = pd.DataFrame(times).T
df_start_end.columns = ["start", "end"]
df_start_end

In [ ]:
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

years = mdates.YearLocator()
months = mdates.MonthLocator()
monthsFmt = mdates.DateFormatter('%b') 
yearsFmt = mdates.DateFormatter('\n\n%Y')  # add some space for the year label

fig, ax = plt.subplots(figsize=(20,4))
for i, floor in enumerate(df_start_end.index):
    start = df_start_end.iloc[i].start
    end = df_start_end.iloc[i].end
    length = end - start
    bar_height = 0.75 # if this is 1.0, the bars vertically touch each other
    ax.broken_barh([(start, length)], (i-bar_height/2, bar_height), color=cycle[i])


# format axis and plot ticks
ax.grid(True)
ax.set_yticks(np.arange(0, df_start_end.shape[0]))
ax.set_yticklabels(df_start_end.index.to_list())
ax.xaxis.remove_overlapping_locs = False # https://github.com/tacaswell/matplotlib/commit/d3b948400500c4adbc9c88903913b1ec054358be
ax.minorticks_on()
ax.xaxis.set_minor_locator(months)
ax.xaxis.set_minor_formatter(monthsFmt)
plt.setp(ax.xaxis.get_minorticklabels(), rotation=90)
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)

# Crop 
left, right = plt.xlim()
plt.xlim(left=left+50, right=right-50)

plt.tight_layout()

fpath = os.path.expanduser(f"~/dropbox/ml4icu/metadata-and-qc/heart-center-bm-first-and-last-timestamps{IMAGE_EXT}")
plt.savefig(fpath, dpi=150)
print(f"Saved {fpath}")